In [22]:
#encoding=utf-8
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
# import lightgbm as lgb
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB

import pickle

from all_model_py import CutDebt, IDClassifier, IfKnowDebtor, Installment, WillingToPay, ConfirmLoan

# import sys
# sys.path.append('../CutDebt/')
# sys.path.append('../IDClassifier/')
# sys.path.append('../IfKnowDebtor/')
# sys.path.append('../Installment/')
# sys.path.append('../SetDueDay/')
# sys.path.append('../WillingToPay/')

# from CutDebt_py import CutDebt
# from IDClassifier_py import IDClassifier
# from IfKnowDebtor_py import IfKnowDebtor
# from Installment_py import Installment
# from SetDueDay_py import SetDueDay
# from WillingToPay_py import WillingToPay

In [23]:
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
func_list = [CutDebt,IDClassifier,IfKnowDebtor,Installment,ConfirmLoan,WillingToPay]

ind = 0
for each_model in model_list:
    print(each_model)
    
    # load data
    path = '../../data/{}/'
    data_train = pd.read_csv(path.format(each_model) + 'cleaned_mock_up_data.csv', encoding='utf8')
    print(data_train.shape)

    
    # get tfidf
    phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

    print('fitting phrase')
    phrase_vectorizer.fit(data_train.split_text)

    print('transform phrase')
    phrase = phrase_vectorizer.transform(data_train.split_text)

    
    # linear svc
    l_svc = LinearSVC()
    lsvc = CalibratedClassifierCV(l_svc) 
    lsvc.fit(phrase, data_train.label)
    
    
    # logistic
    log_r = LogisticRegression()
    log_r.fit(phrase, data_train.label)
    
    
    # Naive Bayes
    naive_b = MultinomialNB()
    naive_b.fit(phrase, data_train.label)
    
    
    print('finish training')
    
    
    
    
    
#     for threhold
    
    # save model
    save_path = '../../savedModel/{}/'
#     # save tfidf
#     pickle.dump(phrase_vectorizer, open(save_path.format(each_model) + "tfidf.pickle", "wb"))
#     # save linear svc
#     pickle.dump(lsvc, open(save_path.format(each_model) + "LinearSVC.pickle", "wb"))
#     # save logistic
#     pickle.dump(log_r, open(save_path.format(each_model) + "Logistic.pickle", "wb"))
#     # save lightGBM
# #     pickle.dump(lgbm_model, open(save_path.format(each_model) + "Lgbm.pickle", "wb"))
#     # save naive bayes
#     pickle.dump(naive_b, open(save_path.format(each_model) + "nb.pickle", "wb"))

    
    other = pickle.load(open('../../savedModel/others/{}/{}_other.pickle'.format(each_model,each_model),'rb'))
    
    func = func_list[ind]
    result = func(svc=lsvc, logistic=log_r, nb=naive_b, tfidf=phrase_vectorizer, other=other,  jieba_path='../WordCut/userdict.txt')
    pickle.dump(result, open(save_path.format(each_model) + each_model + '.pickle', "wb"))
    ind = ind + 1
    
    
    
    

CutDebt
(8341, 2)
fitting phrase
transform phrase
finish training
IDClassifier
(3992, 2)
fitting phrase
transform phrase


ValueError: Requesting 3-fold cross-validation but provided less than 3 examples for at least one class.

In [9]:
# import jieba
# sentence = '好'
# sentence = jieba.cut(sentence)
# sentence = ' '.join(sentence)
# sentence = phrase_vectorizer.transform([sentence])
# naive_b.predict_proba(sentence)

In [10]:
idc = pickle.load(open("../../savedModel/IDClassifier/IDClassifier.pickle", 'rb'))
cutd = pickle.load(open("../../savedModel/CutDebt/CutDebt.pickle", 'rb'))
ifk = pickle.load(open("../../savedModel/IfKnowDebtor/IfKnowDebtor.pickle", 'rb'))
will = pickle.load(open("../../savedModel/WillingToPay/WillingToPay.pickle", 'rb'))
inst = pickle.load(open("../../savedModel/Installment/Installment.pickle", 'rb'))
conf = pickle.load(open("../../savedModel/ConfirmLoan/ConfirmLoan.pickle", 'rb'))

In [20]:
idc.classify('我不认识')

{'av_pred': array([ 0.62810537,  0.2913421 ,  0.08055253]),
 'label': 0,
 'pred_prob': array([[ 0.60713515,  0.35841661,  0.03444824],
        [ 0.55666789,  0.30445086,  0.13888125],
        [ 0.72051308,  0.21115882,  0.0683281 ]])}